In [1]:
import pandas as pd

pd.options.display.max_columns= 100
pd.options.display.max_rows= 1000

In [2]:
#Read in persisted data created in previous question
all_large_ship_journies = pd.read_csv('data/all_large_ship_journies.csv')

#Read in other sources
fleet = pd.read_csv('data/fleet_reg.csv')
portlog = pd.read_excel('data/portlog.xlsx')
vsl_pos = pd.read_csv('data/vsl_pos_170101_170531.csv')
bci = pd.read_csv('data/bci_170101_170531.csv')

In [3]:
#Get Brazilian ports
port_list = ['PontaDaMadeira', 'Tubarao', 'PontaUbu', 'PortoAcu', 'ItaguaiGuaiba']

# 2. Proportion of Iron into Brazil From North Atlantic vs South Africa/Indian Ocean

## Results

1040 ships from Brazil total
* 533 from South Africa/Indian Ocean ~51%
* 162 from North Atlantic ~16%
* 140 from other areas Brazil, Colombia, Trinidad ~ 13%

* Remainder of departure ports have no assigned country code

### Caveats
* Only regestering ports in the portlog. Seem to be a lot missing. e.g. Durban lat long (-29.8697	31.0324	)
    * More of a problem for Q2

* If a ship docked and left in less than 24 hours this would be missed

* Assumes journey data always appears chronologically (seems to be the case)

In [4]:
all_large_ship_journies.head()

,index,imo,date,lat,lon,speed,draft,eta,destination,status,port
0,10732,9701712,2017-01-01 02:43:53.000000,24.1352,52.7777,0.00000,6.8,2017-02-03 12:00:00.000000,SANTOS_BRAZIL,NaN,NaN
1,14622,9701712,2017-01-01 23:21:12.000000,24.1353,52.7777,0.00000,10.8,2017-02-02 12:00:00.000000,SANTOS_BRAZIL,NaN,NaN
2,19242,9701712,2017-01-02 00:01:28.000000,24.1353,52.7777,0.00000,10.8,2017-02-02 12:00:00.000000,SANTOS_BRAZIL,NaN,NaN
3,31136,9701712,2017-01-02 23:46:50.000000,24.7458,52.7350,9.55874,10.4,2017-02-02 12:00:00.000000,SANTOS_BRAZIL,NaN,NaN
4,31170,9701712,2017-01-03 00:06:23.000000,24.7530,52.7994,11.29670,10.4,2017-02-02 12:00:00.000000,SANTOS_BRAZIL,NaN,NaN


In [5]:
#As in q1 all ships that arrived and left brazil over 60,000dwt
from_brazil = all_large_ship_journies.loc[
    (all_large_ship_journies['port'].isin(port_list)) & (all_large_ship_journies['status'] == 'left')]

In [6]:
# How many journies?
journies = len(from_brazil)
print(journies)

1040


In [7]:
#Ships that arrived and left Brazil
brazil_ships = from_brazil.imo.unique()

In [8]:
#Journies of ships that went to Brazil
brazil_ship_journies = all_large_ship_journies.loc[all_large_ship_journies['imo'].isin(brazil_ships)]

In [9]:
brazil_ship_journies.head()

,index,imo,date,lat,lon,speed,draft,eta,destination,status,port
2864,8142,9681508,2017-01-01 00:35:24.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang
2865,13705,9681508,2017-01-01 23:54:28.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang
2866,20053,9681508,2017-01-02 00:49:30.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang
2867,24697,9681508,2017-01-02 23:09:26.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang
2868,24963,9681508,2017-01-03 00:03:23.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang


In [10]:
#Flag where the ship departed from before arriving at next port
#Again really slow. Definitely better way
#Can't think of better implementation due to referencing different rows
#If you want to see the result a csv is available in the Google Drive folder I shared with you.
# 'data/brazil_ship_last_ports.csv'

brazil_ship_last_ports = pd.DataFrame()
i=0
print(len(brazil_ships))
for ship in brazil_ships:
    ship_journey = brazil_ship_journies.loc[brazil_ship_journies['imo']==ship]
    ship_journey.reset_index(inplace=True, drop=True)
    ship_journey['port'] = ship_journey['port'].astype(str)
    ship_journey['last_port'] = ''
    last_port = ''
    print(i)
    i = i + 1
    for x in range(0,len(ship_journey)):
        if ship_journey['port'][x] != 'nan':
            last_port = ship_journey['port'][x]
            ship_journey['last_port'][x+1] =  last_port
        else:
            ship_journey['last_port'][x+1] =  last_port
    brazil_ship_last_ports = brazil_ship_last_ports.append(ship_journey)
    
    

798
0


/home/tom/app/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tom/app/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tom/app/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [49]:
brazil_ship_last_ports.to_csv('data/brazil_ship_last_ports.csv', index=False)

In [46]:
brazil_ship_last_ports.head()

,index,imo,date,lat,lon,speed,draft,eta,destination,status,port,last_port
0,8142,9681508,2017-01-01 00:35:24.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang,
1,13705,9681508,2017-01-01 23:54:28.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang,Xingang
2,20053,9681508,2017-01-02 00:49:30.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang,Xingang
3,24697,9681508,2017-01-02 23:09:26.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang,Xingang
4,24963,9681508,2017-01-03 00:03:23.000000,39.0499,117.7633,0.0,6.5,2016-12-28 20:00:00.000000,TIANJIN CHINA,NaN,Xingang,Xingang


In [17]:
departure_ports = brazil_ship_last_ports.loc[
    (brazil_ship_last_ports['status'] == 'arrived') &
    (brazil_ship_last_ports['port'].isin(port_list))]
departure_ports = departure_ports.merge(portlog, how='left', left_on='last_port', right_on='name')

In [37]:
departure_countries = departure_ports['country'].value_counts()

In [32]:
#Malaysia, China, India, Taiwan, Australia, Oman, South Africa, Japan, South Korea, Bahrain, Mozambique, Saudi
#Philippines, Qatar, Thailand

SA_IO_ships = (
    departure_countries['MA'] + departure_countries['CH'] + departure_countries['IN'] + departure_countries['TA'] +
    departure_countries['AU'] + departure_countries['OM'] + departure_countries['SO'] + departure_countries['JA'] +
    departure_countries['KO'] + departure_countries['BA'] + departure_countries['MO'] + departure_countries['SA'] +
    departure_countries['PH'] + departure_countries['QA'] + departure_countries['TH']
    )
    
    
#Netherlands, Spain, US, France, Turkey, Italy, Germany, Denmark, UK, Slovenia, Canada, Croatia, Russia, Mexico
#Portugal, Ireland, Romania, Norway    
NA_ships =(
    departure_countries['NE'] + departure_countries['SP'] + departure_countries['US'] + departure_countries['FR'] +
    departure_countries['TU'] + departure_countries['IT'] + departure_countries['GE'] + departure_countries['DE'] +
    departure_countries['UK'] + departure_countries['SL'] + departure_countries['CA'] + departure_countries['CR'] +
    departure_countries['RU'] + departure_countries['ME'] + departure_countries['PO'] + departure_countries['IR'] +
    departure_countries['RO'] + departure_countries['NO']
    )
    
#Brazil, Colombia, Trinidad    
other_ships =(
    departure_countries['BR'] + departure_countries['CO'] + departure_countries['TR']
    )

In [35]:
print(SA_IO_ships)
print(NA_ships)
print(other_ships)
print(SA_IO_ships+NA_ships+other_ships)

533
162
140
835
